---
# Demo Use Case (Part 1)

A data scientist is part of a team developing an advanced driver-assistance system (ADAS).
They continuously collect video, lidar, and other sensor data from their fleet of test vehicles.
The data scientist wants to test a new object detection model on video collected today.
They specifically want to test how the model performs in identifying pedestians.

The purpose of this Jupyter notebook is to simulate the ingestion of real-time video into
Streaming Data Platform (SDP). 
To avoid the need for a set of live cameras for this demo, we play back video
from a set of MP4 files.

---

### Install dependencies

See [install_dependencies.ipynb](install_dependencies.ipynb).

### How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, opencv_image_to_mpl
import cv2
from matplotlib import pyplot as plt

imp.reload(video);

### Define Pravega stream parameters

In [ ]:
gateway = 'pravega-grpc-gateway.examples.frightful-four.eaglemonk.intranet.nautilus-platform-dev.com:80'
scope = 'examples'
stream = 'virat'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

### Copy from a video file to Pravega

In [ ]:
!ls ../../data/virat/ground/VIRAT_S_000200_01_000226_000268.mp4

In [ ]:
# filename = '../../data/SampleVideo_1280x720_5mb.mp4'
#filename = 'DJI_0039.MP4'  # video with detected trucks
# filename = '../../data/DJI_0023.MP4'  # 
filename = '../../data/virat/ground/VIRAT_S_000200_01_000226_000268.mp4'
output_stream = OutputStream(pravega_client, scope, stream)
output_stream.create_stream()

In [ ]:
#output_stream.truncate_stream()

In [ ]:
# Crop images - left, top, right, bottom
# crop = [int(c) for c in [4096*1/6,2160*1/6,4096*5/6,2160*5/6]]
crop = None
#size = (600,400)
size = None
fps = 1
crop, size

In [ ]:
%%time
output_stream.write_video_from_file(filename, crop=crop, size=size, fps=fps)

### Copy from a video file in a loop

In [ ]:
%%time
while True:
    output_stream.write_video_from_file('DJI_0039.MP4', crop=[int(c) for c in [4096*1/6,2160*1/6,4096*5/6,2160*5/6]], size=(600,400), fps=8)
    output_stream.write_video_from_file('DJI_0023.MP4', crop=[int(c) for c in [4096*1/6,2160*1/6,4096*5/6,2160*5/6]], size=(600,400), fps=8)
    output_stream.write_video_from_file('cars.mp4', crop=None, size=(600,400), fps=8)
    output_stream.write_video_from_file('Pexels Videos 2103099.mp4', crop=None, size=(600,400), fps=8)
    output_stream.write_video_from_file('Tourist Crossing The Street.mp4', crop=None, size=(600,400), fps=8)
    output_stream.write_video_from_file('video.mp4', crop=None, size=(600,400), fps=8)

# Get head and tail stream cuts

In [ ]:
unindexed_stream = UnindexedStream(pravega_client, scope, stream)
stream_info = unindexed_stream.get_stream_info()
stream_info

In [ ]:
stream_size_MB = (list(stream_info.tail_stream_cut.cut.values())[0] - list(stream_info.head_stream_cut.cut.values())[0]) * 1e-6
stream_size_MB

# Playground

In [ ]:
vidcap = cv2.VideoCapture(filename)
video_frames = output_stream.opencv_video_frame_generator(vidcap)

vidcap.get(cv2.CAP_PROP_POS_FRAMES)

vidcap.get(cv2.CAP_PROP_FPS)

next(video_frames)['image'].shape

cropped_video_frames = (output_stream.cropped_video_frame(f, crop) for f in video_frames)
resized_video_frames = (output_stream.resized_video_frame(f, size) for f in cropped_video_frames)
events_to_write = output_stream.video_frame_write_generator(resized_video_frames)

img = next(resized_video_frames)['image']
fig = plt.figure(figsize = (12,8))
plt.imshow(opencv_image_to_mpl(img), interpolation='none');

# evt = next(events_to_write)
# evt.event[0:100]

# len(next(events_to_write).event)